<a href="https://colab.research.google.com/github/mimrancomsats/ProgrammingforAI_FALL24/blob/main/Lab_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Quiz-2**

Perform following tasks on the provided Reviews Dataset.
* Drop words if not alphabets.
* Tokenize the sentence.
* Perform lemitization.
* Vectorize using bigram and trigram techniques.
* Apply Random Forest algorithm with 150 trees.
* Evaluate overall accuracy of the model and class-wise precision .

In [29]:
import numpy as np
import pandas as pd
import string


import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')


stopwords.words('english')
exclude = string.punctuation

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split


In [53]:
df = pd.read_csv('./reviews_dataset.csv')

In [54]:
df.head()

,news,type
0,China had role in Yukos split-up\n \n China le...,business
1,Oil rebounds from weather effect\n \n Oil pric...,business
2,Indonesia 'declines debt freeze'\n \n Indonesi...,business
3,$1m payoff for former Shell boss\n \n Shell is...,business
4,US bank in $515m SEC settlement\n \n Five Bank...,business


In [55]:
df.shape

(2225, 2)

In [56]:
unique_classes = df['type'].unique()
print("Unique classes in 'type':")
print(unique_classes)

Unique classes in 'type':
['business' 'entertainment' 'politics' 'sport' 'tech']


# Text Preprocessing

In [57]:
def tokenize_text(text):
    return word_tokenize(text)

def remove_non_alpha(tokens):
    return [word for word in tokens if word.isalpha()]

def lemmatize_words(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word.lower()) for word in tokens]


df['news'] = df['news'].str.lower()

df['news'] = df['news'].apply(tokenize_text)

df['news'] = df['news'].apply(remove_non_alpha)

df['news'] = df['news'].apply(lemmatize_words)

print(df)

                                                   news      type
0     [china, had, role, in, yukos, china, lent, rus...  business
1     [oil, rebound, from, weather, effect, oil, pri...  business
2     [indonesia, debt, indonesia, no, longer, need,...  business
3     [payoff, for, former, shell, bos, shell, is, t...  business
4     [u, bank, in, sec, settlement, five, bank, of,...  business
...                                                 ...       ...
2220  [microsoft, launch, it, own, search, microsoft...      tech
2221  [warning, about, junk, mail, deluge, the, amou...      tech
2222  [microsoft, get, the, blogging, bug, software,...      tech
2223  [gamers, snap, up, new, sony, psp, gamers, hav...      tech
2224  [apple, laptop, is, the, apple, powerbook, ha,...      tech

[2225 rows x 2 columns]


In [58]:
df['news'] = df['news'].apply(lambda x: ' '.join(x))

# Encoding the target Variable

In [59]:
from sklearn.preprocessing import LabelEncoder

X = df['news']
Y = df['type']

print(X)
print(Y)


0       china had role in yukos china lent russia to h...
1       oil rebound from weather effect oil price reco...
2       indonesia debt indonesia no longer need the de...
3       payoff for former shell bos shell is to pay to...
4       u bank in sec settlement five bank of america ...
                              ...                        
2220    microsoft launch it own search microsoft ha un...
2221    warning about junk mail deluge the amount of s...
2222    microsoft get the blogging bug software giant ...
2223    gamers snap up new sony psp gamers have bought...
2224    apple laptop is the apple powerbook ha been ch...
Name: news, Length: 2225, dtype: object
0       business
1       business
2       business
3       business
4       business
          ...   
2220        tech
2221        tech
2222        tech
2223        tech
2224        tech
Name: type, Length: 2225, dtype: object


In [60]:
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

print(Y)

[0 0 0 ... 4 4 4]


# Splitting the dataset

In [61]:

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

print(X_train.shape)
print(X_test.shape)


(1780,)
(445,)


# Bi-Gram

In [48]:
cv = CountVectorizer(ngram_range=(2, 2))
X_train_n_gram = cv.fit_transform(X_train)
X_test_n_gram = cv.transform(X_test)

print(f"Shape of X_train_n_gram: {X_train_n_gram.shape}")
print(f"Shape of X_test_n_gram: {X_test_n_gram.shape}")

rf = RandomForestClassifier(n_estimators=150, random_state=42)

rf.fit(X_train_n_gram, y_train)

y_pred = rf.predict(X_test_n_gram)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Shape of X_train_n_gram: (1780, 252894)
Shape of X_test_n_gram: (445, 252894)
Accuracy: 0.8697
Confusion Matrix:
[[106   1   3   2   3]
 [ 10  51   0  10   1]
 [  4   1  65   6   0]
 [  0   0   0 102   0]
 [  9   2   0   6  63]]

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       115
           1       0.93      0.71      0.80        72
           2       0.96      0.86      0.90        76
           3       0.81      1.00      0.89       102
           4       0.94      0.79      0.86        80

    accuracy                           0.87       445
   macro avg       0.89      0.85      0.87       445
weighted avg       0.88      0.87      0.87       445



# Tri-gram

In [47]:
cv = CountVectorizer(ngram_range=(3, 3))
X_train_n_gram = cv.fit_transform(X_train)
X_test_n_gram = cv.transform(X_test)

print(f"Shape of X_train_n_gram: {X_train_n_gram.shape}")
print(f"Shape of X_test_n_gram: {X_test_n_gram.shape}")

rf = RandomForestClassifier(n_estimators=150, random_state=42)
rf.fit(X_train_n_gram, y_train)

y_pred = rf.predict(X_test_n_gram)

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Shape of X_train_n_gram: (1780, 480193)
Shape of X_test_n_gram: (445, 480193)
Overall Accuracy: 0.7640
Confusion Matrix:
[[ 94   0   4  16   1]
 [  5  38   0  27   2]
 [  2   1  59  14   0]
 [  0   1   0 101   0]
 [  7   2   1  22  48]]

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.82      0.84       115
           1       0.90      0.53      0.67        72
           2       0.92      0.78      0.84        76
           3       0.56      0.99      0.72       102
           4       0.94      0.60      0.73        80

    accuracy                           0.76       445
   macro avg       0.84      0.74      0.76       445
weighted avg       0.83      0.76      0.77       445



# Applying Bi-Gram and Tri-Gram

In [62]:
cv = CountVectorizer(ngram_range=(2, 3))
X_train_n_gram = cv.fit_transform(X_train)
X_test_n_gram = cv.transform(X_test)

print(f"Shape of X_train_n_gram: {X_train_n_gram.shape}")
print(f"Shape of X_test_n_gram: {X_test_n_gram.shape}")

rf = RandomForestClassifier(n_estimators=150, random_state=42)
rf.fit(X_train_n_gram, y_train)

y_pred = rf.predict(X_test_n_gram)

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Shape of X_train_n_gram: (1780, 733087)
Shape of X_test_n_gram: (445, 733087)
Overall Accuracy: 0.8629
Confusion Matrix:
[[109   0   4   2   0]
 [ 12  46   1  13   0]
 [  4   0  65   7   0]
 [  0   0   0 102   0]
 [ 11   1   0   6  62]]

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.95      0.87       115
           1       0.98      0.64      0.77        72
           2       0.93      0.86      0.89        76
           3       0.78      1.00      0.88       102
           4       1.00      0.78      0.87        80

    accuracy                           0.86       445
   macro avg       0.90      0.84      0.86       445
weighted avg       0.88      0.86      0.86       445

